In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from matplotlib import pyplot as plt 
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
from shapely.geometry import Point

In [ ]:
import importlib
import traitement_donnees
importlib.reload(traitement_donnees)
from traitement_donnees import *

import dataFrameUtil
importlib.reload(dataFrameUtil)
from dataFrameUtil import *

# Partie 1 : Préparation des données

## 1.1 Nettoyage et synthétisation de la base de données "revenus" avec une CAH

In [ ]:
revenus = getRevenus()
revenus.head()

La base de données "revenus" contient des informations sur les revenus disponibles des ménages au sein de chaque IRIS, dont notamment les déciles de revenus, des informations sur les inégalités des revenus et sur la composition des revenus (revenu d'activité, du patrimoine, des prestations sociales...). A partir de cette base de données, nous avons décider de réaliser une classification ascendante hiérarchique (CAH) pour répondre à plusieurs objectifs :  
- Diminuer le nombre de variables à analyser : la base de données contient 29 colonnes et réaliser une CAH permet d'avoir une information qui synthétise l'ensemble des variables pertinentes pour l'analyse plutôt que de les analyser chacune individuellement. Construire un profil économique plutôt que d'analyser chaque variable indépendamment permet d'avoir une vision plus globale de chaque IRIS.
- Cette méthode peut relever des interactions entre variables et donc permettre de dégager des effets d'interaction ou des effets non linéaires lorsque les clusters construits seront utilisés dans des analyses de régressions.
- Avoir une unique variable catégorielle pour représenter le profil économique des quartiers est utile pour la visualisation de données.

In [ ]:
print(revenus.shape)
revenus.isna().sum()

Cette étape nous permet de repérer que la colonne tp60 (taux de pauvreté) contient un nombre important de valeurs manquantes (presque un quart des quartiers). Nous retirerons cette colonne pour notre CAH. Les autres colonnes contiennent toutes un nombre relativement faible de valeurs manquantes. On imputera une valeur (la médiane) à la place de ces valeurs manquantes.

Certaines colonnes doivent aussi être retirées pour réaliser la CAH, notamment la colonne "iris", qui contient le code de l'IRIS et n'indique rien sur le profil économique du quartier, de même que la variable "note" qui donne de l'information sur la qualité des données et non sur le profil économique du quartier.

En raison de la forte redondance entre les différentes variables sur le revenu disponible qui pourraient effacer le rôle des inégalités de revenu lors de l'analyse, nous enlèverons également une partie des données sur les revenus des différents déciles. Nous garderons tout de même les revenus des déciles 1, 5 et 9 et des quartiles 1 et 3.

In [ ]:
rev_cah = getRevenusCah(revenus)

La base "rev_cah" est une sous-base de la base "revenus" qui conserve uniquement les colonnes utiles pour la CAH. Dans cette base, les valeurs manquantes ont été imputées par la valeur médiane et les variables ont été standardisées afin d'éviter qu'une dimension domine la classification hiérarchique.

In [ ]:
# réalisation de la CAH
cah = doCAH(rev_cah)

plotDendrogramme(cah)

Pour notre analyse, nous avons besoin d'un nombre suffisamment grand de cluster pour identifier différents types de cluster (minimum 4), mais pas trop important pour ne pas trop compliquer l'analyse. Pour déterminer combien de cluster garder, nous utilisons la méthode du coude.

In [ ]:
plotCoude(cah)

On utilise la distance des fusions pour tracer une courbe et identifier le “coude”, c’est-à-dire le nombre de clusters où fusionner davantage devient peu utile.

Conserver 5 clusters plutôt que 4 permet de conserver plus d'informations, mais entre 5 et 7 clusters, la distance de fusion est proche. Garder 8 clusters nous semble un nombre trop important pour notre analyse. Nous avons donc décidé de conserver 5 clusters.

In [ ]:
# Découpage en clusters
createClusters(revenus, cah)

summary_with_total = analyzeClusters(revenus)

print(summary_with_total.to_string())


Il s'agit ici d'analyser la manière dont les quartiers ont été regroupés au sein de chaque clusters. On observe la moyenne de certaines variables pour effectuer cette analyse : 
- le taux de pauvreté
- des indicateurs sur le revenu : médiane (med), quartile 1 (q1) et quartile 3 (q3)
- des indicateurs sur les inégalités : rapport interdéciles (rd) et indice de gini (gi)
- la part des revenus d'activité dans le revenu disponible (pact)
- la part des salaires et traitements dans les revenus d'activité (ptsa), dont la part des indemnités de chômage dans les revenus d'activité (pcho), la part des revenus des activités non salariées dans les revenus d'activité (pben) et la part des pensions, retraites et rentes dans les revenus d'activités (ppen) 
- la part des revenus du patrimoine dans le revenu disponible (ppat)
- la part des prestations sociales dans le revenu disponible (ppsoc) dont la part des minima sociaux dans les prestations sociales (ppmini).

En termes de salaire, on observe qu'aussi bien pour le revenu médian que les revenus des quartiles 1 et 3, l'ordre des quartiers des plus pauvres au plus riche et le suivant : 1, 2, 3, 5, 4. On retrouve ce même ordre concernant le taux de pauvreté, la part des revenus du patrimoine, des prestations sociales, des minima sociaux et des impôts.

Ce qui rapproche le cluster 4 du cluster 3 et qui pourrait expliquer pourquoi l'ordre des clusters 4 et 5 n'est pas inversé, ce sont les variables sur la part des revenus d'activité dans le revenu disponible et la part des salaires et traitements dans les revenus d'activité. Cependant, lorsque l'on analyse les autres sources de revenus, on remarque que cette proximité du taux des revenus d'activité s'explique différemment. Le niveau plus faible de la part des revenus d'activité des quartiers du cluster 4 s'explique part l'importance des revenus du patrimoine, ce qui n'est pas le cas des quartiers du cluster 3. Similairement, la part moins importante des traitements et salaires dans les revenus d'activité des quartiers du cluster 4 tiennent en partie à la place importante des revenus des activités non salariés, contrairement aux quartiers du cluster 3 pour lesquels ces revenus n'ont pas une place importante.

Ces différentes observations nous permettent de conclure que, même si les quartiers du cluster 4 sont plus proches selon la CAH des quartiers 3 que les quartiers 5, on peut tout de même analyser les clusters selon le niveau de richesse des quartiers, dans l'ordre établi précédemment (1, 2, 3, 5, 4).
Cela correspondant également aux inégalités au sein des quartiers : les clusters avec des quartiers plus pauvres sont aussi les clusters avec des quartiers plus inégalitaires et réciproquement.

In [ ]:
renameClusters(revenus)

# nombre de quartiers par clusters
print(revenus["cluster_label"].value_counts())

Les clusters "très riches" sont les moins nombreux, ce qui semble assez cohérent avec différentes analyses sur le décrochage des très hauts revenus et qui peuvent se retrouver au niveau des quartiers (cette explication est peut-être bancale).

Les quartiers "moyens" sont les plus nombreux. Comme ils se situent au milieu concernant les données et qu'ils sont les plus nombreux, ils serviront de référence lors des différentes analyses qui nécessitent une modalité de référence.

## 1.2 Fusion des bases

In [ ]:
iris = getIris(revenus)

La fonction getIris permet de fusionner plusieurs bases (iris, population et revenus). Cela permet d'avoir dans une même base les données géographiques, démographiques et économiques des quartiers. Cependant, les bases de données ne correspondent pas exactement car les données économiques sur les IRIS ne sont pas disponibles pour tous les IRIS. En outre, les codes IRIS changent légèrement d'une année sur l'autre et certaines correspondances ne se font donc pas. Il existe des tables de passage pour associer les bases de données IRIS selon les années, mais elles ne sont pas disponibles pour les années les plus récentes. Il s'agit alors de vérifier que ces problèmes dans la fusion des bases de données ne posera pas de problème pour la suite de l'analyse.

In [ ]:
print("Pourcentage de valeurs manquantes concernant certaines variables suite à la fusion des bases :")
print(iris.isna()[["code_iris", "cluster_label", "pop"]].mean()*100)

La plupart des IRIS n'ont pas de données économiques associées. C'est assez logique, puisque la base avec les données géographiques des IRIS contient plus de 48 000 lignes, alors que la base "revenus" n'en contient que 12 000. 
Seulement 8% des IRIS n'ont pas de données démographiques associées.

In [ ]:
print("Pourcentage de valeurs manquantes concernant le profil économique des IRIS selon leur type :")
print(iris.groupby("type_iris_label")["cluster_label"].apply(lambda x: x.isna().mean() * 100))

print("\nNombre d'IRIS de chaque type :")
print(iris["type_iris_label"].value_counts())

On remarque que les IRIS de type "autres" n'ont pas de données économiques disponibles, de même que la plupart des IRIS "activité" et "divers", c'est-à-dire les grandes zones spécifiques peu habitées et ayant une superficie importante (parcs de loisirs, zones portuaires, forêts, ...). En revanche, les IRIS "habitat" ont presque tous des valeurs concernant les revenus des habitants. Les données de l'INSEE disponibles concernent probablement seulement les quartiers qui ont beaucoup d'habitants pour des raisons de protection des données personnelles. Comme la majorité des IRIS sont de type "autre", il manque beaucoup de données sur le profil économique des quartiers. Pour qu'on puisse utiliser ces données par la suite, il faudra vérifier que les formations post-bac se trouvent majoritairement dans des quartiers pour lesquelles ces données sont disponibles.

In [ ]:
print("Pourcentage de valeurs manquantes concernant le nombre d'habitants selon le type d'IRIS :")
iris.groupby("type_iris_label")["pop"].apply(lambda x: x.isna().mean()*100)

Ce sont principalement les iris "autres" qui ont le moins souvent des données. On n'a pas de correspondance parfaite non plus pour les autres types d'IRIS, probablement en raison des changements des IRIS entre les différentes bases de données selon l'année de référence. 

In [ ]:
parcoursup_total = getParcoursup(iris)

Après avoir associé à chaque formation un IRIS et les données économiques et démographiques qui lui sont associées, on peut vérifier si les IRIS dans lesquels se trouvent les formations sont fortement affectés par le manque de données sur le profil économique et démographique des IRIS.

In [ ]:
# vérifications
print(parcoursup_total.isna()[["code_iris", "cluster_label", "pop"]].sum())
print(parcoursup_total.shape)

762 formations n'ont pas été associées à des IRIS. Cela tient probablement au fait qu'une partie des formations de parcoursup ne se trouvent pas en France.

La plupart des formations ont des données sur socio-démographiques sur leur quartier. 3249 formations sur les 14079 n'ont pas de données sur le profil économique du quartier. On a donc ces données pour 77% des formations, ce qui est suffisant pour mener des analyses.

In [ ]:
iris = updateIris(iris, parcoursup_total)

On ajoute à la base "iris" le fait qu'il y ait ou non au moins une formation dans l'IRIS et le fait qu'il y ait ou non au moins une formation sélective.

# Partie 2 : Problématisation sur l'inégale mobilité des étudiants

Nous faisons l'hypothèse que les étudiants dont la famille a de revenus faibles n'ont pas les mêmes possibilités de mobilité durant leurs études que les étudiants issus de familles aisées.

La base de données "parcoursup" ne permet pas de vérifier cela directement puisque les données sont agrégées au niveau des formations. On ne peut donc pas savoir si les boursiers sont aussi mobiles que les autres étudiants ou non.

On peut toutefois observer à l'échelle agrégée la corrélation entre la part de boursiers dans la formation et la part d'étudiants qui viennent de la même académie que la formation. 

In [ ]:
plotParcoursupMobilitesBoursiers(parcoursup_total)

In [ ]:
parcoursup_xy = parcoursup_total[["admis_boursier", "part_memeac2"]].dropna()
pearsonr(parcoursup_xy["admis_boursier"], parcoursup_xy["part_memeac2"])

Les formations qui attirent la plus d'étudiants de leur académie sont aussi celles qui contiennent le plus grand nombre de boursiers. Ainsi, les 10% de formations qui ont le plus d'étudiants issus de la même académie ont 29% d'étudiants boursiers, contre 18% pour les 10% de formations qui ont le moins d'étudiants issus de la même académie. La part d'étudiants issus d'une académie différente de celle de la formation post-bac est significativement différente selon la part de boursiers dans la formation. Cela peut être lié au fait que les boursiers vont plus souvent dans des formations proches car ils ont moins les moyens financiers d'avoir leur propre logement et donc d'aller dans des formations éloignées. 

Dès lors, l’étude de la localisation des formations apparaît essentielle, car la mobilité des étudiants varie fortement selon leur profil socio-économique. Dans la mesure où certains étudiants sont plus mobiles que d'autres, la répartition géographique de l’offre de formation n’est pas neutre : elle influence directement l’accès aux études post-bac et le choix qui s'offre à chaque étudiant. Il est ainsi crucial d’analyser les mécanismes qui déterminent l’implantation des formations afin d’évaluer dans quelle mesure l’offre de formation est réellement équitable pour l’ensemble des étudiants.

Notre projet a ainsi pour but d'étudier la localisation des formations post-bac et d'en analyser les déterminants pour essayer de comprendre dans quelle mesure la localisation des formations participe aux inégalités d'accès aux différentes filières de l'enseignement supérieur.

# Partie 3 : Les formations ne sont pas réparties aléatoirement en France

## 3.1 Profil économique des quartiers et présence d'une formation

In [ ]:
# Carte de Lyon
printCarteVille(iris, parcoursup_total, code_dept="69", center=(45.7640, 4.8357), zoom=12)

on voit que les formations se trouvent surtout dans les quartiers moyens, moins souvent dans les pauvres et jamais dans les très pauvres. Les quartiers riches sont souvent excentrés et sans formation aussi.

In [ ]:
# Carte de Nantes
printCarteVille(iris, parcoursup_total, code_dept="44", center=(47.2184, -1.5536), zoom=11)

même observations que pour lyon

In [ ]:
pd.crosstab(iris["cluster_label"], iris[NB_FORMATIONS], normalize="index", dropna=False)

à l'échelle nationale : 21% des quartiers moyens ont une formation, 19% des pauvres, 13% des riches... = pas la même proportion selon le type de quartier

## 3.2 Profil démographique des quartiers et présence d'une formation

In [ ]:
plotFormationDemographie(iris, bin_col="pop_bin", value_col=NB_FORMATIONS,
                         xlabel="Nombre d'habitants", ylabel="Proportion de formations",
                         title="Probabilité que le quartier contienne une formation selon le nombre d'habitants")

Plus il y a d'habitants dans un quartier, plus la probabilité qu'il y ait une formation semble élevée. Cette probabilité augemente particulièrement entre 1000 et 3000 habitants : elle est très faible avant 1000 habitants et est difficile à interpréter pour les quartiers au-delà de 3000 habitants en raison de leur faible nombre.

In [ ]:
printCarteFranceMetropolitaine(iris, parcoursup_total)

les formations sont concentrés dans les grandes villes et il y a des zones assez grandes avec très peu de formation (par exemple : centre-val de loire en dehors de Orléan et Tours, bourgogne en dehors de Dijon)

In [ ]:
plotFormationDemographie(iris, bin_col="part1824_bin", value_col=NB_FORMATIONS,
                         xlabel="Part des 18–24 ans (%)", ylabel="Probabilité d'avoir une formation",
                         title="Probabilité d'avoir une formation selon la part des 18–24 ans")

plus la part des 18-24 est élevée, plus il y a de chances qu'il y ait une formation dans le quartier. Mais question du sens de la causalité : parce que ce sont des quartiers avec beaucoup de jeunes, des formations se sont installées, ou parce qu'il y a des formations, des jeunes sont venus vivre dans le quartier alors qu'ils vivaient dans un autre quartier avant ? pour le vérifier, on peut observer la part des 11-17 ans

In [ ]:
plotFormationDemographie(iris, bin_col="part1117_bin", value_col=NB_FORMATIONS,
                         xlabel="Part des 11–17 ans (%)", ylabel="Proportion de formations",
                         title="Proportion de formations selon la part des 11–17 ans")

Les quartiers avec beaucoup de jeunes (11-17 ans) sont souvent les quartiers les plus pauvres, qui sont ceux qui n'ont pas souvent de formations, ce qui explique pourquoi les formations ne se trouvent pas dans les quartiers avec beaucoup de jeunes.

Contrairement à ce à quoi on pourrait s'attendre, il n'y a pas plus de formation dans les quartiers avec beaucoup d'adolescentes (ici les 11 à 17 ans). En revanche, il y a plus de formations dans les quartiers avec beaucoup de jeunes 18-24 ans, probablement parce que ces quartiers attirent les 18-24 (et non pas parce que les formations sont localisées dans les quartiers où il y avait déjà beaucoup de jeunes).

## 3.3 Modélisation de la probabilité qu'un quartier possède une formation

In [ ]:
constructLogitModel(df=iris, y=NB_FORMATIONS, terms=logit_terms)

toutes choses égales par ailleurs (nombre d'habitants, type d'IRIS), on retrouve à peu près les mêmes résultats que dans le tableau des tris croisés sur profil économique / proba d'avoir une formation. Aussi, on remarque que les quartiers avec + d'habitants ont + de chances d'avoir une formation (effet qui semble faible mais c'est parce que c'est l'effet d'un seul habitant supplémentaire)

# Partie 4 :  L'inégale répartition des formations sélectives

## 4.1 Définir les formations sélectives

In [ ]:
print(parcoursup_total["selectivite"].value_counts())
(parcoursup_total["taux_acces"] < 50).mean()

on a choisi de redéfinir les formations sélectives car la manière de définir sélectif pour parcoursup c'est toutes les formations qui n'ont pas 100% comme taux d'accès. On a choisit de définir comme sélectif les formations avec moins de 50% de taux d'accès (ie 38% des formations)

## 4.2 Profil socio-économique des quartiers et présence d'une formation sélective

In [ ]:
# Carte de Paris
printCarteVilleSelect(iris, parcoursup_total, code_depts=["75"], center=(48.8566, 2.3522), zoom_start=12)

beaucoup de formations dans les quartiers bleus foncés, mais aussi des formations sélectives dans les quartiers orange = pas de tendance claire

In [ ]:
# Carte de Strasbourg
printCarteVilleSelect(iris, parcoursup_total, code_depts=["67"], center=(48.583, 7.745), zoom_start=12)

pareil : les formations ne sont pas dans les quartiers très pauvres, mais on n'observe pas clairement de différence dans la répartition formation sélective / non sélective entre les quartiers moyens et pauvres

In [ ]:
pd.crosstab(iris["cluster_label"], iris[TRES_SELECT], normalize="index", dropna=False)

ici on voit que les quartiers très riches ont plus souvent une formation très sélective et les quartiers très pauvres moins souvent 

In [ ]:
plotFormationDemographie(iris, bin_col="pop_bin", value_col=TRES_SELECT,
                         xlabel="Nombre d'habitants", ylabel="Proportion de formations sélectives",
                         title="Proportion de formations sélectives selon le nombre d'habitants du quartier")

Plus il y a d'habitants dans un quartier, plus la probabilité qu'il y ait une formation sélective semble élevée.

## 4.3 Modélisation de la probabilité qu'un quartier possède une formation sélective

In [ ]:
constructLogitModel(df=iris, y=TRES_SELECT, terms=logit_terms)

comparer les coefficients avec ceux de la régression sur le fait d'avoir une formation pour expliquer pourquoi on fait la deuxième régression après

Interprétation des résultats du modèle logit :

Le tableau ci-dessus présente les résultats d’un modèle logit estimant la probabilité pour un quartier (IRIS) d’accueillir au moins une formation sélective. La variable dépendante est binaire et vaut 1 si l’IRIS comporte au moins une formation sélective, 0 sinon. Le modèle est estimé par maximum de vraisemblance sur un échantillon de 11 424 IRIS et a convergé correctement.

Qualité globale du modèle:

Le test du rapport de vraisemblance (LLR p-value < 10⁻¹²⁰) permet de rejeter très nettement l’hypothèse nulle selon laquelle l’ensemble des coefficients seraient nuls. Le modèle explique donc significativement la présence de formations sélectives. Le pseudo-R² s’élève à environ 5,8 %, ce qui est un niveau courant pour un modèle logit appliqué à des données spatiales et suggère que, bien que le modèle capte une part non négligeable des déterminants, une fraction importante de la localisation des formations reste expliquée par des facteurs non observés.

Lecture des coefficients associés aux types de quartiers:

Les coefficients estimés pour les types de quartiers sont exprimés en *log-odds* et doivent être interprétés relativement à une catégorie de référence. Dans ce modèle, les quartiers de type *moyen* constituent la catégorie de référence. Ainsi, chaque coefficient mesure l’écart de probabilité d’accueillir une formation sélective entre un type de quartier donné et un quartier moyen, toutes choses égales par ailleurs.

Un coefficient négatif indique que le type de quartier considéré a une probabilité plus faible que les quartiers moyens d’accueillir une formation sélective, tandis qu’un coefficient positif indique une probabilité plus élevée. Pour faciliter l’interprétation, ces coefficients peuvent être exponentiés afin d’obtenir des *odds ratios*, qui indiquent le facteur multiplicatif des chances relatives par rapport à la catégorie de référence.

Effet du type socio-économique du quartier:

Les quartiers *pauvres* présentent une probabilité significativement plus faible que les quartiers moyens d’accueillir une formation sélective. Cet écart est encore plus prononcé pour les quartiers *très pauvres*, dont la probabilité est fortement réduite par rapport à celle des quartiers moyens.
À l’autre extrémité de la distribution, les quartiers *très riches* se distinguent nettement des quartiers moyens par une probabilité significativement plus élevée d’accueillir une formation sélective. L’exponentiation du coefficient associé montre que les chances relatives d’implantation y sont plus de deux fois supérieures à celles observées dans les quartiers moyens. En revanche, les quartiers *riches* ne diffèrent pas significativement des quartiers moyens au seuil de 5 %, suggérant que l’avantage spatial se concentre principalement dans les quartiers les plus favorisés, et non de manière monotone avec le niveau de richesse.

Ces résultats révèlent une forte non-linéarité des inégalités socio-spatiales : les quartiers très riches concentrent une part disproportionnée de l’offre sélective, tandis que les quartiers pauvres et très pauvres sont nettement désavantagés.

Effet du type fonctionnel de l’IRIS:

Le modèle contrôle également pour le type fonctionnel des IRIS. À caractéristiques socio-économiques comparables, les IRIS à dominante résidentielle (*habitat*) présentent une probabilité plus faible d’accueillir une formation sélective que les IRIS d’activité, ce qui reflète une logique d’implantation liée à la présence d’infrastructures universitaires et de pôles d’enseignement.

Variables de contrôle démographiques et sociales:

Parmi les variables continues, la population âgée de 18 à 24 ans a un effet positif et fortement significatif : les quartiers comptant davantage de jeunes adultes ont une probabilité plus élevée d’accueillir des formations sélectives, ce qui est cohérent avec une logique de proximité à la population étudiante.

Le revenu médian apparaît avec un coefficient négatif conditionnellement aux clusters de quartiers, ce qui suggère que l’effet du niveau de revenu est déjà largement capturé par la typologie socio-économique globale. De même, la part des cadres et la part des prestations sociales dans le revenu ne présentent pas d’effet significatif une fois ces typologies prises en compte, indiquant une redondance informationnelle avec les clusters.


## 4.4 Modélisation de la probabilité qu'une formation soit sélective selon les caractéristiques du quartier

In [ ]:
constructLogitModel(df=parcoursup_total, y=TRES_SELECT, terms=logit_terms)

# Partie 5 : La localisation de la formation influence le nombre de boursiers

## 5.1 Modélisation de la part de boursiers par formation selon le quartier

faire des stats sur le fait que les élèves ne sont pas forcément mobiles : question de la mixité sociale des formations ? / de l'accès aux formations 

In [ ]:
parcoursup_total["admis_boursier"].quantile([0.25, 0.333, 0.5, 0.667, 0.75])

In [ ]:
parcoursup_total.groupby("cluster_label")["admis_boursier"].mean()

In [ ]:
plotBoursierQuartier(parcoursup_total)

In [ ]:
print(constructModelBoursier(parcoursup_total, terms=terms_boursier1).summary())

## 5.2 Modélisation de la part de boursier dans une formation en contrôlant par le type de formation

On retrouve que les formations dans les quartiers les plus pauvres ont plus de boursiers.

In [ ]:
print(constructModelBoursier(parcoursup_total, terms=terms_boursier2).summary())

Si on contrôle par le type de formation, les formations dans les quartiers riches n'ont pas moins de boursiers (en proportion) que les quartiers "moyens".

In [ ]:
pd.crosstab(parcoursup_total["cluster_label"], parcoursup_total["type_form"], normalize="index", dropna=False)

Les formations dans les quartiers riches et très riches sont plus souvent des CPGE et des écoles de commerce et moins souvent des BTS. Les formations dans les quartiers dits très riches sont plus souvent des écoles d'ingénieurs, à l'inverse des quartiers dits très pauvres.

In [ ]:
parcoursup_total.groupby("type_form")["admis_boursier"].mean()

Les formations qui concentrent le plus grand nombre de boursiers sont les BTS, autrement dit les formations que l'on trouve le plus souvent dans des quartiers pauvres. 
Au contraire, les formations qui concentrent le moins de boursiers (écoles de commerce, écoles d'ingénieur, CPGE) se trouvent plus souvent dans des quartiers riches.
La question de la causalité se pose : est-ce que les formations s'installent là où elles attirent des personnes ou est-ce que les personnes vont dans les formations les plus proches ? Question du sens de la causalité.

Globalement, il y a presque systématiquement moins de boursiers dans les quartiers très riches (à l'exception des licences LAS) et plus de boursiers dans les quartiers pauvres et très pauvres que dans les quartiers moyens (ce qui confirme les résultats de la régression linéaire). Les formations dans les quartiers riches concentrent souvent plus de bousiers que les formations des quartiers moyens lorsqu'on contrôle par le type de formation. Ainsi, la richesse du quartier a un effet sur le nombre de boursiers dans les formations mais cet effet n'est pas linéaire : ce sont les quartiers moyens et les quartiers très riches qui concentrent le moins de boursiers, puis les quartiers riches et pauvres, et enfin les quartiers très pauvres dont les formations contiennent le plus de boursiers.

In [ ]:
# Carte de Paris
printCarteTypeFormation(iris, parcoursup_total, deps=["75"], center=(48.8566, 2.3522), zoom=12)

In [ ]:
# Carte de Lille
printCarteTypeFormation(iris, parcoursup_total, deps=["59"], center=(50.632, 3.057), zoom=12)